### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
# langsmith tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [5]:
# Data ingestion - From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.langchain.com/langsmith/prompt-engineering-concepts")
docs = loader.load()
docs


[Document(metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-concepts', 'title': 'Prompt engineering concepts - Docs by LangChain', 'language': 'en'}, page_content='Prompt engineering concepts - Docs by LangChainSkip to main content🚀 Share how you\'re building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationPrompt engineering conceptsGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupOverviewQuickstartConceptsCreate and update promptsCreate a promptManage promptsManage prompts programmaticallyConfigure prompt settingsUse tools in a promptInclude multimodal content in a promptWrite your prompt with AIConnect to modelsTutorialsOptimize a classifierSync prompts with GitHubTest multi-turn conversationsOn this pageWhy prompt engineering?Prompts vs. prompt templatesPrompts in LangSmithChat vs CompletionF-string vs. mustacheToolsStru

In [ ]:
# Load Data --> Docs --> Divide our text into chunks --> text to vectors --> Vector Embedding --> Vectore Store DB

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-concepts', 'title': 'Prompt engineering concepts - Docs by LangChain', 'language': 'en'}, page_content="Prompt engineering concepts - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationPrompt engineering conceptsGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupOverviewQuickstartConceptsCreate and update promptsCreate a promptManage promptsManage prompts programmaticallyConfigure prompt settingsUse tools in a promptInclude multimodal content in a promptWrite your prompt with AIConnect to modelsTutorialsOptimize a classifierSync prompts with GitHubTest multi-turn conversationsOn this pageWhy prompt engineering?Prompts vs. prompt templatesPrompts in LangSmithChat vs CompletionF-string vs. mustacheToolsStruc

In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()


In [9]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents, embeddings)

In [10]:
vectorstoredb

In [11]:
# Query from a vector db
query = "While traditional software applications are built by writing code, AI applications often derive their logic from prompts."
result = vectorstoredb.similarity_search(query)
result[0].page_content

'outputModelPrompt versioningCommitsTagsPrompt playgroundTesting multiple promptsTesting over a datasetVideo guidePrompt engineering conceptsCopy pageCopy pageWhile traditional software applications are built by writing code, AI applications often derive their logic from prompts.'

In [12]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [13]:
# Retrieval Chain, Document chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain



RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000028234F04160>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000028235545E70>, root_client=<openai.OpenAI object at 0x0000028234F047C0>, root_async_client=<openai.AsyncOpenAI object at 0x0000028235547760>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputParser(), k

In [14]:
from langchain_core.documents import Document
document_chain.invoke({
     "input":"While traditional software applications are built by writing code, AI applications often derive their logic from prompts.",
     "context":[Document(page_content="While traditional software applications are built by writing code, AI applications often derive their logic from prompts.This guide will walk through the key concepts of prompt engineering in LangSmith.")]
})


'AI applications often derive their logic from prompts rather than traditional coding according to the provided context.'

In [15]:
# However, we want the documents to first come from the retriver we jast set up. That way, we can use the retriver to dynamically select the most relevant documetns and pass those in for a given question.

In [16]:
# Retriver
vectorstoredb

In [19]:
retriever = vectorstoredb.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriever, document_chain)

In [20]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028235545CC0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | Chat

In [21]:
# Get the resposne from the LLM
response=retriever_chain.invoke({"input":"While traditional software applications are built by writing code, AI applications often derive their logic from prompts."})
response['answer']

'What is the difference between "prompts" and "prompt templates"?\n\nPrompts refer to the messages that are passed into the language model, while prompt templates refer to a way of formatting information to get that prompt to hold the information you want. Prompt templates can include variables for few shot examples, outside context, or any other external data needed in the prompt.'

In [22]:
response

{'input': 'While traditional software applications are built by writing code, AI applications often derive their logic from prompts.',
 'context': [Document(id='4bb4b107-fcc4-4806-bc84-6e8e17d7a265', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-concepts', 'title': 'Prompt engineering concepts - Docs by LangChain', 'language': 'en'}, page_content='outputModelPrompt versioningCommitsTagsPrompt playgroundTesting multiple promptsTesting over a datasetVideo guidePrompt engineering conceptsCopy pageCopy pageWhile traditional software applications are built by writing code, AI applications often derive their logic from prompts.'),
  Document(id='dc63a6f4-8f1c-4cce-ba1e-19bac3e09acb', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-concepts', 'title': 'Prompt engineering concepts - Docs by LangChain', 'language': 'en'}, page_content='We often see that prompt engineering is multi-disciplinary. Sometimes the best prompt engineer is not

In [23]:
response['context']

[Document(id='4bb4b107-fcc4-4806-bc84-6e8e17d7a265', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-concepts', 'title': 'Prompt engineering concepts - Docs by LangChain', 'language': 'en'}, page_content='outputModelPrompt versioningCommitsTagsPrompt playgroundTesting multiple promptsTesting over a datasetVideo guidePrompt engineering conceptsCopy pageCopy pageWhile traditional software applications are built by writing code, AI applications often derive their logic from prompts.'),
 Document(id='dc63a6f4-8f1c-4cce-ba1e-19bac3e09acb', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-concepts', 'title': 'Prompt engineering concepts - Docs by LangChain', 'language': 'en'}, page_content='We often see that prompt engineering is multi-disciplinary. Sometimes the best prompt engineer is not the software engineer who is building the application, but rather the product manager or another domain expert. It is important to have the proper 